In [ ]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.12.0
Eager mode:  True
Hub version:  0.14.0
GPU is NOT AVAILABLE


Upload the all-data.csv file here for the stock data

In [ ]:
uploaded = files.upload()

Saving all-data.csv to all-data (1).csv


Depending on the dataset, run different cells. One cell is for training on imdb movie reviews, other is for training on kaggle stock headline data.

In [ ]:
# Specify dataset by running certain blocks and changing vars here

dataType = 'headlines'
split = 1000

if dataType == 'headlines':

  import pandas as pd
  from google.colab import files
  import io

  all_data = uploaded['all-data (1).csv']
  all_data = b'sentiment,headline\r' + all_data

  df = pd.read_csv(io.StringIO(all_data.decode('ISO-8859-1')))

  replacements = {'positive': 1, 'neutral': 0.5, 'negative': 0}

  df['sentiment'] = df['sentiment'].replace(replacements)
  df.describe()
  df.head(5)

  train_data = df.head(2000)
  test_data = df.head(-2000)

  train_examples = train_data['headline']
  train_labels = train_data['sentiment']
  test_examples = train_data['headline']
  test_labels = train_data['sentiment']
else:

  split = 10000
  train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"],
                                  batch_size=-1, as_supervised=True)

  train_examples, train_labels = tfds.as_numpy(train_data)
  test_examples, test_labels = tfds.as_numpy(test_data)

In [ ]:
model = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 1.57878883e-02,  4.98380885e-02,  2.40863606e-01,
         1.87875345e-01, -3.39158103e-02,  7.67395720e-02,
         1.81269839e-01,  2.77704865e-01, -2.42365032e-01,
         3.05864774e-02, -2.43232727e-01,  1.32182494e-01,
        -7.32430592e-02, -1.00961305e-01,  1.73457235e-01,
        -2.27750972e-01, -8.20414796e-02,  9.01060849e-02,
         1.87368229e-01, -4.32811752e-02,  5.76531403e-02,
        -3.97308134e-02,  2.06172943e-01, -1.47577614e-01,
        -3.43066037e-01,  3.00952107e-01, -1.98043063e-01,
        -1.72513187e-01, -3.14834192e-02, -1.44412473e-01,
        -5.08709475e-02,  4.44432124e-02,  2.47089058e-01,
        -1.61841244e-01, -2.36676067e-01,  1.78138092e-01,
        -1.10819958e-01, -3.44441772e-01,  4.43259403e-02,
        -9.44948196e-02, -4.03841108e-01, -8.07946995e-02,
        -1.83016852e-01,  3.14553738e-01,  2.08202007e-04,
        -2.89767921e-01, -1.58644561e-02, -1.23980902e-01,
       

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 50)                48190600  
                                                                 
 dense_6 (Dense)             (None, 16)                816       
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [ ]:
x_val = train_examples[:split]
partial_x_train = train_examples[split:]

y_val = train_labels[:split]
partial_y_train = train_labels[split:]

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=1,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

2/2 [==============================] - 4s 2s/step - loss: 0.6730 - accuracy: 0.2440 - val_loss: 0.5730 - val_accuracy: 0.7770


In [ ]:
type(model)

keras.engine.sequential.Sequential

Now that model trained, lets grab headlines related to stock ticker.

In [ ]:
import requests
import bs4

In [ ]:
text= "AAPL"
url = 'https://news.google.com/search?q=' + text + '&hl=en-CA&gl=CA&ceid=CA:en'

# Fetch the URL data using requests.get(url),
# store it in a variable, request_result.
request_result=requests.get( url )

# Creating soup from the fetched request
soup = bs4.BeautifulSoup(request_result.text,
                         "html.parser")

heading_object=soup.find_all( 'h3' )

npHeadlines = []

# Iterate through the object
# and print it as a string.
for info in heading_object:
    # print(info.getText())
    # print("------")
    npHeadlines.append(info.getText())

npHeadlines = np.array(npHeadlines)
npHeadlines = npHeadlines[:3]

Now that all headlines for specified company are obtained, lets run the model on this data to perform sentiment analysis.

In [ ]:
preds = model.predict(npHeadlines)

1/1 [==============================] - 0s 221ms/step


In [ ]:
print(preds)
print(npHeadlines)

[[0.29072705]
 [0.44235942]
 [0.5773897 ]]
['Apple (NASDAQ:AAPL) Slips After Settlement News - TipRanks.com'
 'Leaked Info About Apple iPhone Hints At Surprising Changes - Apple (NASDAQ:AAPL)'
 '5 Signs a Market Bounce May Be Imminent']
